In [35]:
import os
import json
import torch
import numpy as np
from tqdm import tqdm
from torch import nn, optim
from torch.utils.data import DataLoader
from models.classifier_lstm import ClassifierLSTM

from importnb import Notebook
with Notebook():
    from load_data import GenderDataset

In [19]:
device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda')

In [30]:
classifier_embedding_size = 512
classifier_hidden_size = 512
classifier_num_layers = 2
classifier_is_bidirectional = True

classifier_lr = 0.001
classifier_num_epoch = 10
classifier_batch_size = 2048

In [11]:
with open(os.path.join(os.getcwd(), 'data', 'blog.json'), 'r') as file:
    json_data = json.load(file)
json_data.keys()

dict_keys(['docs', 'configs'])

In [12]:
docs = json_data['docs']
sample = docs[1113]
print(sample.keys())
from collections import defaultdict
gender_count = defaultdict(int)
for doc in docs:
    gender = doc['gender']
    gender_count[gender] += 1
print(gender_count)

dict_keys(['rawtext', 'author', 'gender', 'age', 'work', 'tokens', 'actage', 'split', 'attrib'])
defaultdict(<class 'int'>, {'male': 9812, 'female': 9865})


In [15]:
tokenized_docs = [" ".join(doc['tokens']) for doc in docs]
gender_label = [int(doc['gender'] == 'male') for doc in docs] # Male 1, Female 0
raw_text = [doc['rawtext'] for doc in docs]

In [18]:
gender_dataset = GenderDataset(tokenized_docs, gender_label, raw_text=raw_text)

In [21]:
gender_dataloader = DataLoader(
    gender_dataset,
    batch_size=classifier_batch_size,
    shuffle=True,
    num_workers=0,
)

In [36]:
classifier_model = ClassifierLSTM(
    len(gender_dataset.idx2token), 
    classifier_embedding_size, 
    classifier_hidden_size, 
    classifier_num_layers, 
    classifier_is_bidirectional
).to(device)

In [33]:
optim = optim.Adam(classifier_model.parameters(), lr = classifier_lr)

In [34]:
criterion = nn.CrossEntropyLoss()

In [ ]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

def train(dataloader, model, criterion, optimizer, num_epoch):
    for epoch in range(num_epoch):
        model.train()
        for batch, (src_ids, src_len, tgt) in tqdm(enumerate(dataloader)):
            optimizer.zero_grad()
            
            packed_src = pack_padded_sequence(src_ids.to(device), src_len, batch_first=True, enforce_sorted=False)
            
